En este notebook utilzaremos la librería web3 para escuchar los log del contrato faucet4 que deployamos en el archivo contract5.sol.

La version de web3.py es la 5.29.0

In [60]:
import json
from web3 import Web3, HTTPProvider #web3 es la libreria que se encarga de todo lo relacionado con interactuar con web3
import time

In [61]:
# instanciamos la red que vamos a usar con su cliente rpc, nosotros usaremos el de rinkeby provisto por infura
provider_rpc = {
    "development": "http://localhost:9933",
    "alphanet": "https://rinkeby.infura.io/v3/9aa3d95b3bc440fa88ea12eaa4456161",
}

web3 = Web3(Web3.HTTPProvider(provider_rpc["alphanet"]))  # seleccionamos la red correcta
print ("Latest Ethereum rinkeby block number" , web3.eth.blockNumber) #para asegurarnos que la conección es correcta imprimimos el número del últmo bloque que se anexó en la cadena


Latest Ethereum rinkeby block number 10592800


In [51]:
#para poder interactuar con la cadena requeriremos del ABI y dirección del contrato, el ABI lo guardamos en el archivo JSON llamado abi.json, y lo cargamos con ayuda de la libreria
#json en la variable abi
with open("/content/abi.json") as f:
    info_json = json.load(f)
abi = info_json 

contractAddress = '0x1C588421e7662B8A45Aa08AB0BC52E27BaAA1D65'
contract = web3.eth.contract(address=contractAddress, abi=abi)#con esto enlazamos con el contrato de nuestro interes

Deposit_Event = contract.events.Deposit() #Definimos el evento que queremos escuchar en nuestro caso el evento deposit
Withdrawal_Event = contract.events.Withdrawal() #Definimos el evento que queremos escuchar en nuestro caso el evento deposit

In [52]:
def handle_event(event):
  """
  Esta función es la encargada de manejar los eventos, es decir es la parte que escucha el contrato desde la blockchain
  """
  receipt = web3.eth.waitForTransactionReceipt(event['transactionHash']) #resivimos el evento del contrato, ie la interacción
  result = Deposit_Event.processReceipt(receipt)#extraemos la información de dicho evento y si es uno de tipo Deposit_Event obtenemos la información
  if result:#este if es solo para evitar un error por si result es vacío porque ejecutamos alguna función con un evento y no sea deposit dicho evento
    print("Deposit:")
    print(result[0]['args'])#imprimimos el contenido del evento, ie la dirección y la cantidad
    return
  result = Withdrawal_Event.processReceipt(receipt)
  print("Withdrawal:")
  print(result[0]['args'])#imprimimos el contenido del evento, ie la dirección y la cantidad

def log_loop(event_filter, poll_interval):
  """
  Esta función es la que se encarga de ejecutar el escaneo en la cadena de las interacciones con el contrato
  """
  while True:#un loop infinito para siempre estar ejecutando
    for event in event_filter.get_new_entries():#para cada interacción con el contrato
      handle_event(event)
      time.sleep(poll_interval)#realizamos una pausa en el escaneo, esto es para no saturar el servicio de infura

block_filter = web3.eth.filter({'fromBlock':10592301, 'address':contractAddress})#definimos que transacciones nos van a importar, las que ocurrieron después del bloque 10592301, el cual
                                                                                 #es el bloque de despliegue del contrato
log_loop(block_filter, 2)#ejecutamos la función de análisis

/usr/local/lib/python3.7/dist-packages/web3/contract.py:1167: UserWarning: The log with transaction hash: HexBytes('0x198eb6621eaadb238310fb0f2fdceccd28cbe783441fc7761e168e8efd43c4e1') and logIndex: 6 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  f"The log with transaction hash: {log['transactionHash']!r} and "


Withdrawal:
AttributeDict({'to': '0x7a46622DfC8E2b61FF166C27576302B6Ae11CF31', 'amount': 1})
Deposit:
AttributeDict({'to': '0x7a46622DfC8E2b61FF166C27576302B6Ae11CF31', 'amount': 1000000000})


KeyboardInterrupt: ignored